In [30]:
import numpy as np
data = """

"""
with open("input_23.txt", 'r') as f:
    data = f.read()

data = [x.split(" ") for x in data.split("\n") if not x == ""]

#data= data.replace("\n", "")

data

[['set', 'b', '57'],
 ['set', 'c', 'b'],
 ['jnz', 'a', '2'],
 ['jnz', '1', '5'],
 ['mul', 'b', '100'],
 ['sub', 'b', '-100000'],
 ['set', 'c', 'b'],
 ['sub', 'c', '-17000'],
 ['set', 'f', '1'],
 ['set', 'd', '2'],
 ['set', 'e', '2'],
 ['set', 'g', 'd'],
 ['mul', 'g', 'e'],
 ['sub', 'g', 'b'],
 ['jnz', 'g', '2'],
 ['set', 'f', '0'],
 ['sub', 'e', '-1'],
 ['set', 'g', 'e'],
 ['sub', 'g', 'b'],
 ['jnz', 'g', '-8'],
 ['sub', 'd', '-1'],
 ['set', 'g', 'd'],
 ['sub', 'g', 'b'],
 ['jnz', 'g', '-13'],
 ['jnz', 'f', '2'],
 ['sub', 'h', '-1'],
 ['set', 'g', 'b'],
 ['sub', 'g', 'c'],
 ['jnz', 'g', '2'],
 ['jnz', '1', '3'],
 ['sub', 'b', '-17'],
 ['jnz', '1', '-23']]

In [31]:
data[10] = ['set', 'e', '105699']
data

[['set', 'b', '57'],
 ['set', 'c', 'b'],
 ['jnz', 'a', '2'],
 ['jnz', '1', '5'],
 ['mul', 'b', '100'],
 ['sub', 'b', '-100000'],
 ['set', 'c', 'b'],
 ['sub', 'c', '-17000'],
 ['set', 'f', '1'],
 ['set', 'd', '2'],
 ['set', 'e', '105699'],
 ['set', 'g', 'd'],
 ['mul', 'g', 'e'],
 ['sub', 'g', 'b'],
 ['jnz', 'g', '2'],
 ['set', 'f', '0'],
 ['sub', 'e', '-1'],
 ['set', 'g', 'e'],
 ['sub', 'g', 'b'],
 ['jnz', 'g', '-8'],
 ['sub', 'd', '-1'],
 ['set', 'g', 'd'],
 ['sub', 'g', 'b'],
 ['jnz', 'g', '-13'],
 ['jnz', 'f', '2'],
 ['sub', 'h', '-1'],
 ['set', 'g', 'b'],
 ['sub', 'g', 'c'],
 ['jnz', 'g', '2'],
 ['jnz', '1', '3'],
 ['sub', 'b', '-17'],
 ['jnz', '1', '-23']]

In [32]:
import sys
import string
def machine(pid=None):
    numSend = 0
    numMul = 0
    #print(f"hello {pid}")
    #sys.stdout.flush()
    if pid:
        registers = {"p": pid}
    else:
        registers = {}
        for x in string.ascii_lowercase[:8]:
            if x == "a":
                registers[x] = 1
            else:
                registers[x] = 0
    print(registers)
    ptr = 0
    while ptr < len(data):    
        print(ptr, registers)
        op, X, *Y = data[ptr]
        Y = Y or None
        if Y:
            Y = Y[0]
            if Y.isalpha():
                Y = registers[Y] #getting register value
            Y = int(Y)
        for x in [x for x in [X, Y] if x]:
            if str(x).isalpha() and not x in registers.keys():
                registers[x] = 0

        if op == "snd":
            #send to other program
            if X in registers.keys():
                X = registers[X]
            cv.acquire()
            #print(f"sending {X}")
            box = sharedBox[(pid+1)%2]
            box.append(X)
            sharedBox[(pid+1)%2] = box
            if pid == 1:
                numSend += 1
            cv.notify()
            cv.release()
        elif op == "set":
            #set value X = val(Y)
            registers[X] = Y  
        elif op == "add":
            #add value X:= val(X) + val(Y)
            registers[X] = registers[X] + Y 
        elif op == "mul":
            #multiply value X:= val(X) * val(Y)
            registers[X] = registers[X] * Y
            numMul += 1
        elif op == "mod":
            #modulo value X:= val(X) % val(Y)
            registers[X] = registers[X] % Y
        elif op == "rcv":
            #receive from box
            cv.acquire()
            box = sharedBox[pid]
            if len(box) == 0:
                if pid == 1:
                    print(numSend)
                cv.wait()
                box = sharedBox[pid]
            registers[X] = box.pop(0)
            #print(f"received {registers[X]}")
            sharedBox[pid] = box
            cv.release()
        elif op == "jgz":
            #jump
            if X in registers.keys():
                X = registers[X]
            if int(X) > 0:
                ptr += Y
                continue
        elif op == "jnz":
            if X in registers.keys():
                X = registers[X]
            if not int(X) == 0:
                print("")
                ptr += Y
                continue
        elif op == "sub":
            registers[X] = registers[X] - Y 
        else:
            print(f"unknown op {op}")
        ptr += 1
    print(numMul)

In [33]:
machine()

{'a': 1, 'b': 0, 'c': 0, 'd': 0, 'e': 0, 'f': 0, 'g': 0, 'h': 0}
0 {'a': 1, 'b': 0, 'c': 0, 'd': 0, 'e': 0, 'f': 0, 'g': 0, 'h': 0}
1 {'a': 1, 'b': 57, 'c': 0, 'd': 0, 'e': 0, 'f': 0, 'g': 0, 'h': 0}
2 {'a': 1, 'b': 57, 'c': 57, 'd': 0, 'e': 0, 'f': 0, 'g': 0, 'h': 0}

4 {'a': 1, 'b': 57, 'c': 57, 'd': 0, 'e': 0, 'f': 0, 'g': 0, 'h': 0}
5 {'a': 1, 'b': 5700, 'c': 57, 'd': 0, 'e': 0, 'f': 0, 'g': 0, 'h': 0}
6 {'a': 1, 'b': 105700, 'c': 57, 'd': 0, 'e': 0, 'f': 0, 'g': 0, 'h': 0}
7 {'a': 1, 'b': 105700, 'c': 105700, 'd': 0, 'e': 0, 'f': 0, 'g': 0, 'h': 0}
8 {'a': 1, 'b': 105700, 'c': 122700, 'd': 0, 'e': 0, 'f': 0, 'g': 0, 'h': 0}
9 {'a': 1, 'b': 105700, 'c': 122700, 'd': 0, 'e': 0, 'f': 1, 'g': 0, 'h': 0}
10 {'a': 1, 'b': 105700, 'c': 122700, 'd': 2, 'e': 0, 'f': 1, 'g': 0, 'h': 0}
11 {'a': 1, 'b': 105700, 'c': 122700, 'd': 2, 'e': 105699, 'f': 1, 'g': 0, 'h': 0}
12 {'a': 1, 'b': 105700, 'c': 122700, 'd': 2, 'e': 105699, 'f': 1, 'g': 2, 'h': 0}
13 {'a': 1, 'b': 105700, 'c': 122700, 'd':

22 {'a': 1, 'b': 105700, 'c': 122700, 'd': 77, 'e': 105700, 'f': 1, 'g': 77, 'h': 0}
23 {'a': 1, 'b': 105700, 'c': 122700, 'd': 77, 'e': 105700, 'f': 1, 'g': -105623, 'h': 0}

10 {'a': 1, 'b': 105700, 'c': 122700, 'd': 77, 'e': 105700, 'f': 1, 'g': -105623, 'h': 0}
11 {'a': 1, 'b': 105700, 'c': 122700, 'd': 77, 'e': 105699, 'f': 1, 'g': -105623, 'h': 0}
12 {'a': 1, 'b': 105700, 'c': 122700, 'd': 77, 'e': 105699, 'f': 1, 'g': 77, 'h': 0}
13 {'a': 1, 'b': 105700, 'c': 122700, 'd': 77, 'e': 105699, 'f': 1, 'g': 8138823, 'h': 0}
14 {'a': 1, 'b': 105700, 'c': 122700, 'd': 77, 'e': 105699, 'f': 1, 'g': 8033123, 'h': 0}

16 {'a': 1, 'b': 105700, 'c': 122700, 'd': 77, 'e': 105699, 'f': 1, 'g': 8033123, 'h': 0}
17 {'a': 1, 'b': 105700, 'c': 122700, 'd': 77, 'e': 105700, 'f': 1, 'g': 8033123, 'h': 0}
18 {'a': 1, 'b': 105700, 'c': 122700, 'd': 77, 'e': 105700, 'f': 1, 'g': 105700, 'h': 0}
19 {'a': 1, 'b': 105700, 'c': 122700, 'd': 77, 'e': 105700, 'f': 1, 'g': 0, 'h': 0}
20 {'a': 1, 'b': 105700, 

20 {'a': 1, 'b': 105700, 'c': 122700, 'd': 143, 'e': 105700, 'f': 1, 'g': 0, 'h': 0}
21 {'a': 1, 'b': 105700, 'c': 122700, 'd': 144, 'e': 105700, 'f': 1, 'g': 0, 'h': 0}
22 {'a': 1, 'b': 105700, 'c': 122700, 'd': 144, 'e': 105700, 'f': 1, 'g': 144, 'h': 0}
23 {'a': 1, 'b': 105700, 'c': 122700, 'd': 144, 'e': 105700, 'f': 1, 'g': -105556, 'h': 0}

10 {'a': 1, 'b': 105700, 'c': 122700, 'd': 144, 'e': 105700, 'f': 1, 'g': -105556, 'h': 0}
11 {'a': 1, 'b': 105700, 'c': 122700, 'd': 144, 'e': 105699, 'f': 1, 'g': -105556, 'h': 0}
12 {'a': 1, 'b': 105700, 'c': 122700, 'd': 144, 'e': 105699, 'f': 1, 'g': 144, 'h': 0}
13 {'a': 1, 'b': 105700, 'c': 122700, 'd': 144, 'e': 105699, 'f': 1, 'g': 15220656, 'h': 0}
14 {'a': 1, 'b': 105700, 'c': 122700, 'd': 144, 'e': 105699, 'f': 1, 'g': 15114956, 'h': 0}

16 {'a': 1, 'b': 105700, 'c': 122700, 'd': 144, 'e': 105699, 'f': 1, 'g': 15114956, 'h': 0}
17 {'a': 1, 'b': 105700, 'c': 122700, 'd': 144, 'e': 105700, 'f': 1, 'g': 15114956, 'h': 0}
18 {'a': 1, '


10 {'a': 1, 'b': 105700, 'c': 122700, 'd': 215, 'e': 105700, 'f': 1, 'g': -105485, 'h': 0}
11 {'a': 1, 'b': 105700, 'c': 122700, 'd': 215, 'e': 105699, 'f': 1, 'g': -105485, 'h': 0}
12 {'a': 1, 'b': 105700, 'c': 122700, 'd': 215, 'e': 105699, 'f': 1, 'g': 215, 'h': 0}
13 {'a': 1, 'b': 105700, 'c': 122700, 'd': 215, 'e': 105699, 'f': 1, 'g': 22725285, 'h': 0}
14 {'a': 1, 'b': 105700, 'c': 122700, 'd': 215, 'e': 105699, 'f': 1, 'g': 22619585, 'h': 0}

16 {'a': 1, 'b': 105700, 'c': 122700, 'd': 215, 'e': 105699, 'f': 1, 'g': 22619585, 'h': 0}
17 {'a': 1, 'b': 105700, 'c': 122700, 'd': 215, 'e': 105700, 'f': 1, 'g': 22619585, 'h': 0}
18 {'a': 1, 'b': 105700, 'c': 122700, 'd': 215, 'e': 105700, 'f': 1, 'g': 105700, 'h': 0}
19 {'a': 1, 'b': 105700, 'c': 122700, 'd': 215, 'e': 105700, 'f': 1, 'g': 0, 'h': 0}
20 {'a': 1, 'b': 105700, 'c': 122700, 'd': 215, 'e': 105700, 'f': 1, 'g': 0, 'h': 0}
21 {'a': 1, 'b': 105700, 'c': 122700, 'd': 216, 'e': 105700, 'f': 1, 'g': 0, 'h': 0}
22 {'a': 1, 'b':

21 {'a': 1, 'b': 105700, 'c': 122700, 'd': 281, 'e': 105700, 'f': 1, 'g': 0, 'h': 0}
22 {'a': 1, 'b': 105700, 'c': 122700, 'd': 281, 'e': 105700, 'f': 1, 'g': 281, 'h': 0}
23 {'a': 1, 'b': 105700, 'c': 122700, 'd': 281, 'e': 105700, 'f': 1, 'g': -105419, 'h': 0}

10 {'a': 1, 'b': 105700, 'c': 122700, 'd': 281, 'e': 105700, 'f': 1, 'g': -105419, 'h': 0}
11 {'a': 1, 'b': 105700, 'c': 122700, 'd': 281, 'e': 105699, 'f': 1, 'g': -105419, 'h': 0}
12 {'a': 1, 'b': 105700, 'c': 122700, 'd': 281, 'e': 105699, 'f': 1, 'g': 281, 'h': 0}
13 {'a': 1, 'b': 105700, 'c': 122700, 'd': 281, 'e': 105699, 'f': 1, 'g': 29701419, 'h': 0}
14 {'a': 1, 'b': 105700, 'c': 122700, 'd': 281, 'e': 105699, 'f': 1, 'g': 29595719, 'h': 0}

16 {'a': 1, 'b': 105700, 'c': 122700, 'd': 281, 'e': 105699, 'f': 1, 'g': 29595719, 'h': 0}
17 {'a': 1, 'b': 105700, 'c': 122700, 'd': 281, 'e': 105700, 'f': 1, 'g': 29595719, 'h': 0}
18 {'a': 1, 'b': 105700, 'c': 122700, 'd': 281, 'e': 105700, 'f': 1, 'g': 105700, 'h': 0}
19 {'a':

...

11 {'a': 1, 'b': 105700, 'c': 122700, 'd': 696, 'e': 105699, 'f': 1, 'g': -105004, 'h': 0}
12 {'a': 1, 'b': 105700, 'c': 122700, 'd': 696, 'e': 105699, 'f': 1, 'g': 696, 'h': 0}
13 {'a': 1, 'b': 105700, 'c': 122700, 'd': 696, 'e': 105699, 'f': 1, 'g': 73566504, 'h': 0}
14 {'a': 1, 'b': 105700, 'c': 122700, 'd': 696, 'e': 105699, 'f': 1, 'g': 73460804, 'h': 0}

16 {'a': 1, 'b': 105700, 'c': 122700, 'd': 696, 'e': 105699, 'f': 1, 'g': 73460804, 'h': 0}
17 {'a': 1, 'b': 105700, 'c': 122700, 'd': 696, 'e': 105700, 'f': 1, 'g': 73460804, 'h': 0}
18 {'a': 1, 'b': 105700, 'c': 122700, 'd': 696, 'e': 105700, 'f': 1, 'g': 105700, 'h': 0}
19 {'a': 1, 'b': 105700, 'c': 122700, 'd': 696, 'e': 105700, 'f': 1, 'g': 0, 'h': 0}
20 {'a': 1, 'b': 105700, 'c': 122700, 'd': 696, 'e': 105700, 'f': 1, 'g': 0, 'h': 0}
21 {'a': 1, 'b': 105700, 'c': 122700, 'd': 697, 'e': 105700, 'f': 1, 'g': 0, 'h': 0}
22 {'a': 1, 'b': 105700, 'c': 122700, 'd': 697, 'e': 105700, 'f': 1, 'g': 697, 'h': 0}
23 {'a': 1, 'b': 1057

14 {'a': 1, 'b': 105700, 'c': 122700, 'd': 770, 'e': 105699, 'f': 1, 'g': 81282530, 'h': 0}

16 {'a': 1, 'b': 105700, 'c': 122700, 'd': 770, 'e': 105699, 'f': 1, 'g': 81282530, 'h': 0}
17 {'a': 1, 'b': 105700, 'c': 122700, 'd': 770, 'e': 105700, 'f': 1, 'g': 81282530, 'h': 0}
18 {'a': 1, 'b': 105700, 'c': 122700, 'd': 770, 'e': 105700, 'f': 1, 'g': 105700, 'h': 0}
19 {'a': 1, 'b': 105700, 'c': 122700, 'd': 770, 'e': 105700, 'f': 1, 'g': 0, 'h': 0}
20 {'a': 1, 'b': 105700, 'c': 122700, 'd': 770, 'e': 105700, 'f': 1, 'g': 0, 'h': 0}
21 {'a': 1, 'b': 105700, 'c': 122700, 'd': 771, 'e': 105700, 'f': 1, 'g': 0, 'h': 0}
22 {'a': 1, 'b': 105700, 'c': 122700, 'd': 771, 'e': 105700, 'f': 1, 'g': 771, 'h': 0}
23 {'a': 1, 'b': 105700, 'c': 122700, 'd': 771, 'e': 105700, 'f': 1, 'g': -104929, 'h': 0}

10 {'a': 1, 'b': 105700, 'c': 122700, 'd': 771, 'e': 105700, 'f': 1, 'g': -104929, 'h': 0}
11 {'a': 1, 'b': 105700, 'c': 122700, 'd': 771, 'e': 105699, 'f': 1, 'g': -104929, 'h': 0}
12 {'a': 1, 'b': 

23 {'a': 1, 'b': 105700, 'c': 122700, 'd': 839, 'e': 105700, 'f': 1, 'g': -104861, 'h': 0}

10 {'a': 1, 'b': 105700, 'c': 122700, 'd': 839, 'e': 105700, 'f': 1, 'g': -104861, 'h': 0}
11 {'a': 1, 'b': 105700, 'c': 122700, 'd': 839, 'e': 105699, 'f': 1, 'g': -104861, 'h': 0}
12 {'a': 1, 'b': 105700, 'c': 122700, 'd': 839, 'e': 105699, 'f': 1, 'g': 839, 'h': 0}
13 {'a': 1, 'b': 105700, 'c': 122700, 'd': 839, 'e': 105699, 'f': 1, 'g': 88681461, 'h': 0}
14 {'a': 1, 'b': 105700, 'c': 122700, 'd': 839, 'e': 105699, 'f': 1, 'g': 88575761, 'h': 0}

16 {'a': 1, 'b': 105700, 'c': 122700, 'd': 839, 'e': 105699, 'f': 1, 'g': 88575761, 'h': 0}
17 {'a': 1, 'b': 105700, 'c': 122700, 'd': 839, 'e': 105700, 'f': 1, 'g': 88575761, 'h': 0}
18 {'a': 1, 'b': 105700, 'c': 122700, 'd': 839, 'e': 105700, 'f': 1, 'g': 105700, 'h': 0}
19 {'a': 1, 'b': 105700, 'c': 122700, 'd': 839, 'e': 105700, 'f': 1, 'g': 0, 'h': 0}
20 {'a': 1, 'b': 105700, 'c': 122700, 'd': 839, 'e': 105700, 'f': 1, 'g': 0, 'h': 0}
21 {'a': 1

10 {'a': 1, 'b': 105700, 'c': 122700, 'd': 906, 'e': 105700, 'f': 1, 'g': -104794, 'h': 0}
11 {'a': 1, 'b': 105700, 'c': 122700, 'd': 906, 'e': 105699, 'f': 1, 'g': -104794, 'h': 0}
12 {'a': 1, 'b': 105700, 'c': 122700, 'd': 906, 'e': 105699, 'f': 1, 'g': 906, 'h': 0}
13 {'a': 1, 'b': 105700, 'c': 122700, 'd': 906, 'e': 105699, 'f': 1, 'g': 95763294, 'h': 0}
14 {'a': 1, 'b': 105700, 'c': 122700, 'd': 906, 'e': 105699, 'f': 1, 'g': 95657594, 'h': 0}

16 {'a': 1, 'b': 105700, 'c': 122700, 'd': 906, 'e': 105699, 'f': 1, 'g': 95657594, 'h': 0}
17 {'a': 1, 'b': 105700, 'c': 122700, 'd': 906, 'e': 105700, 'f': 1, 'g': 95657594, 'h': 0}
18 {'a': 1, 'b': 105700, 'c': 122700, 'd': 906, 'e': 105700, 'f': 1, 'g': 105700, 'h': 0}
19 {'a': 1, 'b': 105700, 'c': 122700, 'd': 906, 'e': 105700, 'f': 1, 'g': 0, 'h': 0}
20 {'a': 1, 'b': 105700, 'c': 122700, 'd': 906, 'e': 105700, 'f': 1, 'g': 0, 'h': 0}
21 {'a': 1, 'b': 105700, 'c': 122700, 'd': 907, 'e': 105700, 'f': 1, 'g': 0, 'h': 0}
22 {'a': 1, 'b': 

KeyboardInterrupt: 

In [ ]:
b = 57
c = 57
debug:
    b = 57 * 100 = 5700
    b = 5700 - -100000
    c = 5700 - -100000
    c = 5700 - -100000 - -17000
loop:
    f = 1
    d = 2
    e = 2
    g = 2
8 ['set', 'f', '1'],
9 ['set', 'd', '2'],
10     ['set', 'e', '2'],
11         ['set', 'g', 'd'],
12         ['mul', 'g', 'e'],
13         ['sub', 'g', 'b'],
14         ['jnz', 'g', '2'], if g == 0 => f == 0
15             ['set', 'f', '0'],
16         ['sub', 'e', '-1'],
17         ['set', 'g', 'e'],
18         ['sub', 'g', 'b'],
19         ['jnz', 'g', '-8'],
20     ['sub', 'd', '-1'],
21     ['set', 'g', 'd'],
22     ['sub', 'g', 'b'],
23     ['jnz', 'g', '-13'],
24 ['jnz', 'f', '2'], if f == 0 => h += 1
25     ['sub', 'h', '-1'],
26 ['set', 'g', 'b'],
27 ['sub', 'g', 'c'],
28 ['jnz', 'g', '2'], exit if not g == 0
29 ['jnz', '1', '3'],
30 ['sub', 'b', '-17'],
31 ['jnz', '1', '-23']

In [39]:
b = 105700
c = 122700
f = 1
d = 2
h = 0
while not b == c:
    print(b,c)
    while not d == b:
        e = 2
        while not e == b:
            if (d * e) == b:
                f = 0
            e += 1
        d += 1
    if not f:
        h += 1
    b += 17
print(h)

is_prime(b)    

105700 122700


KeyboardInterrupt: 

In [36]:
b % 17

11

In [40]:
from math import sqrt
from itertools import count, islice

def is_prime(n):
    return n > 1 and all(n % i for i in islice(count(2), int(sqrt(n) - 1)))

In [43]:
is_prime(4)

False

In [44]:
is_prime(b)

False

In [51]:
start = 105700
stop = 122700
num_list = []
while not start > stop:
    num_list.append(start)
    start += 17

In [54]:
prime_list = [is_prime(x) for x in num_list]

In [57]:
print(sum(prime_list))
print(len(num_list) - sum(prime_list))

86
915
